<a href="https://colab.research.google.com/github/imxj/imxj.github.io/blob/master/colabs/llms/jax_gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import

In [25]:
from jax import device_put
import time

import jax.numpy as jnp
from jax import lax
import jax
import jax.random as random

import jax.numpy as jnp
from jax import random
from jax.nn.initializers import normal
import flax.linen as nn
import jax.nn as jnn

import optax
from optax import adam

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

### load data

In [26]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-01 21:55:14--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-04-01 21:55:14 (34.1 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [27]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [28]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [29]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [30]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [31]:
'!' in chars

True

In [32]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [33]:
# # let's now encode the entire text dataset and store it into a torch.Tensor
# import torch # we use PyTorch: https://pytorch.org
# data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

# Assuming `encode` is a function defined elsewhere
data = jnp.array(encode(text), dtype=jnp.int64)
print(data.shape, data.dtype)
print(data[:1000])

<ipython-input-33-35845be36863>:8: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in array is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  data = jnp.array(encode(text), dtype=jnp.int64)


(1115394,) int32
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0  0 13 50 50 10  0 35 43  1 49 52 53 61  5
 58  6  1 61 43  1 49 52 53 61  5 58  8  0  0 18 47 56 57 58  1 15 47 58
 47 64 43 52 10  0 24 43 58  1 59 57  1 49 47 50 50  1 46 47 51  6  1 39
 52 42  1 61 43  5 50 50  1 46 39 

#### split into train and test

In [34]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [35]:
val_data.shape

(111540,)

In [36]:
block_size = 8
train_data[:block_size+1]

Array([18, 47, 56, 57, 58,  1, 15, 47, 58], dtype=int32)

#### build feature input x and target output y

In [37]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    print(t)
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

0
when input is [18] the target: 47
1
when input is [18 47] the target: 56
2
when input is [18 47 56] the target: 57
3
when input is [18 47 56 57] the target: 58
4
when input is [18 47 56 57 58] the target: 1
5
when input is [18 47 56 57 58  1] the target: 15
6
when input is [18 47 56 57 58  1 15] the target: 47
7
when input is [18 47 56 57 58  1 15 47] the target: 58


In [38]:
prng = jax.random.PRNGKey(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
ix = random.randint(random.PRNGKey(0), (batch_size,), 0, len(data) - block_size)

def get_batch(split, subkey):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    t1 = time.time()
    ix = random.randint(subkey, (batch_size,), 0, len(data) - block_size)
    t2 = time.time()


    # x = jnp.stack([data[i:i+block_size] for i in ix])
    # y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    def slice_data(i):
        return jax.lax.dynamic_slice(data, (i,), (block_size,))

    x = jax.vmap(slice_data)(ix)
    y = jax.vmap(slice_data)(ix+1)
    x, y = device_put(x), device_put(y)
    # print('TIME rand idx', t2-t1)
    # print('TIME rand idx fetch', time.time()-t2)
    return x, y

xb, yb = get_batch('train', prng)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
(4, 8)
[[ 1 51 39 52 58 50 43  0]
 [25 17 27 10  0 27  6  1]
 [47 51  8  0 14 59 58  1]
 [ 1 57 59 41 46  1 50 43]]
targets:
(4, 8)
[[51 39 52 58 50 43  0 53]
 [17 27 10  0 27  6  1 50]
 [51  8  0 14 59 58  1 46]
 [57 59 41 46  1 50 43 52]]
----
when input is [1] the target: 51
when input is [1, 51] the target: 39
when input is [1, 51, 39] the target: 52
when input is [1, 51, 39, 52] the target: 58
when input is [1, 51, 39, 52, 58] the target: 50
when input is [1, 51, 39, 52, 58, 50] the target: 43
when input is [1, 51, 39, 52, 58, 50, 43] the target: 0
when input is [1, 51, 39, 52, 58, 50, 43, 0] the target: 53
when input is [25] the target: 17
when input is [25, 17] the target: 27
when input is [25, 17, 27] the target: 10
when input is [25, 17, 27, 10] the target: 0
when input is [25, 17, 27, 10, 0] the target: 27
when input is [25, 17, 27, 10, 0, 27] the target: 6
when input is [25, 17, 27, 10, 0, 27, 6] the target: 1
when input is [25, 17, 27, 10, 0, 27, 6, 1] the target: 5

In [39]:
print(xb) # our input to the transformer

[[ 1 51 39 52 58 50 43  0]
 [25 17 27 10  0 27  6  1]
 [47 51  8  0 14 59 58  1]
 [ 1 57 59 41 46  1 50 43]]


### Baseline model: Bigram

In [40]:
import jax
import jax.numpy as jnp
from jax import random
from flax import linen as nn

class BigramLanguageModel(nn.Module):
    vocab_size: int

    @nn.compact
    def __call__(self, idx, targets=None):
        # Token embedding table
        embedding_table = nn.Embed(num_embeddings=self.vocab_size, features=self.vocab_size)
        logits = embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = -jnp.sum(jax.nn.one_hot(targets, C) * jax.nn.log_softmax(logits), axis=1).mean()

        return logits, loss

    def generate(self, params, key, idx, max_new_tokens):
      for _ in range(max_new_tokens):
          logits, _ = self.apply(params, idx)
          logits = logits[:, -1, :]  # (B, C)
          # probs = jax.nn.softmax(logits, axis=-1)  # (B, C)
          key, subkey = random.split(key)
          idx_next = random.categorical(subkey, logits)[:, None]  # (B, 1)
          idx = jnp.concatenate((idx, idx_next), axis=1)  # (B, T+1)
      return idx

# Example usage
model = BigramLanguageModel(vocab_size)

# Initialize model parameters and optimizer
key = random.PRNGKey(1337)
params = model.init(key, jnp.ones((1, 1), jnp.int32))
print(vocab_size, params)
print(params['params']['Embed_0']['embedding'].shape)

flax_apply_jitted = jax.jit(lambda params, xb, yb: model.apply(params, xb, yb))

logits, loss = flax_apply_jitted(params, xb, yb)
print(logits.shape)
print(loss)


65 {'params': {'Embed_0': {'embedding': Array([[-0.07564811, -0.24239814, -0.01499225, ..., -0.06269587,
         0.07931701,  0.1008973 ],
       [-0.00872193,  0.02849229, -0.08602703, ...,  0.12625487,
        -0.05664039, -0.12900828],
       [-0.01677619, -0.01286294, -0.00534049, ..., -0.056512  ,
        -0.11744383, -0.09810068],
       ...,
       [ 0.15148896,  0.08473317, -0.10937848, ...,  0.03070055,
        -0.00960146, -0.15743323],
       [ 0.05090765,  0.06334479, -0.07453259, ..., -0.06299953,
        -0.09558795,  0.02108589],
       [ 0.08294208,  0.0413576 , -0.10926365, ...,  0.05151561,
        -0.18575938,  0.19301337]], dtype=float32)}}}
(65, 65)
(32, 65)
4.1973753


In [41]:
a = jnp.ones([10, 20, 30])
t1 = time.time()
a = a[:, -1, :]
print('TIME total', time.time()-t1)

TIME total 0.008388519287109375


In [89]:
@jax.jit
def slice_fn(x):
  return x[:, -1, :]


In [97]:
a = jnp.ones([10, 99, 30])

In [98]:
%time slice_fn(a).block_until_ready()

CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 674 µs


Array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1

In [99]:
b = jnp.ones([10, 100, 30])

In [100]:
%time slice_fn(b).block_until_ready()

CPU times: user 48.4 ms, sys: 1.02 ms, total: 49.4 ms
Wall time: 50.6 ms


Array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1

In [68]:
t1 = time.time()
for i in range(100):
  a = jnp.ones([10, 100, 30])
  a = slice_fn(a)
print(f'total:{time.time()-t1}')

total:0.10582447052001953


In [50]:
def generate(params, flax_apply_jitted, key, idx, max_new_tokens):
  for _ in range(max_new_tokens):
      t1 = time.time()
      logits, _ = flax_apply_jitted(params, idx, None)
      t2 = time.time()
      print(logits.shape)
      logits = slice_fn(logits)  # (B, C)
      print(logits.shape)
      t3 = time.time()
      key, subkey = random.split(key)
      idx_next = random.categorical(subkey, logits)[:, None]  # (B, 1)
      t4 = time.time()
      idx = jnp.concatenate((idx, idx_next), axis=1)  # (B, T+1)
      print(f'apply {t2-t1}, fetch {t3-t2}, rand: {t4-t3} sample:{time.time()-t4}, total:{time.time()-t1}')
  return idx


In [51]:
t1=time.time()
print(decode(generate(jax.lax.stop_gradient(params), flax_apply_jitted, key, jnp.zeros((1, 1), jnp.int32), 100)[0].tolist()))
print('TIME total', time.time()-t1)

(1, 1, 65)
(1, 65)
apply 0.00012230873107910156, fetch 0.041161537170410156, rand: 0.0023279190063476562 sample:0.0003173351287841797, total:0.043931007385253906
(1, 2, 65)
(1, 65)
apply 0.00048065185546875, fetch 0.04120588302612305, rand: 0.0021431446075439453 sample:0.0002779960632324219, total:0.04410958290100098
(1, 3, 65)
(1, 65)
apply 0.00047898292541503906, fetch 0.03672361373901367, rand: 0.0021305084228515625 sample:0.00027441978454589844, total:0.039609670639038086
(1, 4, 65)
(1, 65)
apply 0.0004787445068359375, fetch 0.03933238983154297, rand: 0.002025127410888672 sample:0.0002868175506591797, total:0.04212522506713867
(1, 5, 65)
(1, 65)
apply 0.0004420280456542969, fetch 0.03999972343444824, rand: 0.0023453235626220703 sample:0.0002808570861816406, total:0.04306983947753906
(1, 6, 65)
(1, 65)
apply 0.000579833984375, fetch 0.03841257095336914, rand: 0.002195596694946289 sample:0.00031304359436035156, total:0.0415036678314209
(1, 7, 65)
(1, 65)
apply 0.0005831718444824219, 

In [2]:
%%timeit

print(decode(generate(jax.lax.stop_gradient(params), flax_apply_jitted, key, jnp.zeros((1, 1), jnp.int32), 100)[0].tolist()))
print('TIME total', time.time()-t1)

NameError: name 'decode' is not defined

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from flax import linen as nn
from optax import adam

# Define the optimizer
learning_rate = 1e-3  # Adjust as needed
tx = adam(learning_rate)

# Initialize model parameters and optimizer state
prng = random.PRNGKey(1337)
params = model.init(prng, jnp.ones((1, 1), jnp.int32))
opt_state = tx.init(params)

# Loss function (assuming you have a batch of data: xb, yb)
def loss_fn(params, xb, yb):
    print(xb, yb)
    logits, loss = model.apply(params, xb, yb)
    return loss

# Update function for a single training step
@jax.jit
def update_step(params, opt_state, xb, yb):
    loss, grads = jax.value_and_grad(loss_fn)(params, xb, yb)
    updates, opt_state = tx.update(grads, opt_state, params)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state, loss

# Training loop (example)
batch_size = 32
for steps in range(100):
    t1 = time.time()
    prng, subkey = random.split(prng)
    xb, yb = get_batch('train', subkey)
    t2 = time.time()
    params, opt_state, loss = update_step(params, opt_state, xb, yb)
    print(f"Epoch {steps},Loss: {loss}, {t2-t1}, {time.time()-t2}")

Traced<ShapedArray(int32[32,32])>with<DynamicJaxprTrace(level=1/0)> Traced<ShapedArray(int32[32,32])>with<DynamicJaxprTrace(level=1/0)>
Epoch 0,Loss: 4.176700115203857, 0.03257417678833008, 0.5034492015838623
Epoch 1,Loss: 4.168147563934326, 0.01342916488647461, 0.0006744861602783203
Epoch 2,Loss: 4.17624568939209, 0.010905742645263672, 0.0005877017974853516
Epoch 3,Loss: 4.169358253479004, 0.010186195373535156, 0.0005640983581542969
Epoch 4,Loss: 4.175686359405518, 0.010203123092651367, 0.0005354881286621094
Epoch 5,Loss: 4.170393943786621, 0.009913444519042969, 0.0005452632904052734
Epoch 6,Loss: 4.169347763061523, 0.013239383697509766, 0.0005314350128173828
Epoch 7,Loss: 4.162715911865234, 0.01039743423461914, 0.0005896091461181641
Epoch 8,Loss: 4.157233238220215, 0.010396957397460938, 0.0005331039428710938
Epoch 9,Loss: 4.163628101348877, 0.00969386100769043, 0.0005295276641845703
Epoch 10,Loss: 4.16217565536499, 0.010032415390014648, 0.0005445480346679688
Epoch 11,Loss: 4.16450309

In [ ]:
print(decode(model.generate(params, key, jnp.ones((1, 1), jnp.int32), 600)[0].tolist()))

 yof le' nongo myorrthavemandse;
SETHOMA of of rerdse ou wilavemerdss s meng l l hinn conotlint w'ds rr, re w'd, r, w'dst w'sthavese.

KIOMPHOMonth gou o l yof thou thar sen lle
GLOMA corrd s w' re;

Yorix's w's
Tdsilit erdss gof thariave rononond, ther o six's
Yore;


By t s
KIASo's yorrr, co s at wix' wiatheco oriaromavemen yofer, shine t.
Whe
Byore y se owrr gortharorint.

Whiarenof ange t
Mo marilbifecou fecour wix's t
POMASoul ou y yotle.
POUCET:
Mof y s y meveve
SEYorthix' yore ser, ss w' tlard'seveve sinof arenour rof rilave you s wrthard,Z' t winou mard'se.
MASoiavecoreathinonomaverd, f


In [ ]:
print(decode(model.generate(params, key, jnp.zeros((1, 1), jnp.int32), 600)[0].tolist()))



Th le' nongo myorrthavemandse;
SETHOMA of of rerdse ou wilavemerdss s meng l l hinn conotlint w'ds rr, re w'd, r, w'dst w'sthavese.

KIOMPHOMonth gou o l yof thou thar sen lle
GLOMA corrd s w' re;

Yorix's w's
Tdsilit erdss gof thariave rononond, ther o six's
Yore;


By t s
KIASo's yorrr, co s at wix' wiatheco oriaromavemen yofer, shine t.
Whe
Byore y se owrr gortharorint.

Whiarenof ange t
Mo marilbifecou fecour wix's t
POMASoul ou y yotle.
POUCET:
Mof y s y meveve
SEYorthix' yore ser, ss w' tlard'seveve sinof arenour rof rilave you s wrthard,Z' t winou mard'se.
MASoiavecoreathinonomaverd, f


In [ ]:
prng = jax.random.PRNGKey(1337)
prng, subkey = random.split(prng)
random.randint(prng, (batch_size,), 0, len(data) - block_size)

Array([ 336148,  278170,  604967,  333936,  240920,  353408,  986691,
        855404, 1063794,  383313,  110829,  521517,  196487,  814541,
        350061,  368494,  978407,  308935,  132008,  539608,  749456,
        375898,  515910,  961126,  283819,  830869,  218387,  546800,
        248480,  867346,  969181,  108388], dtype=int32)

In [ ]:
prng, subkey = random.split(prng)
random.randint(prng, (batch_size,), 0, len(data) - block_size)

Array([ 213304,  267921,  260855,  747830,  768672,  544391,  187257,
        212117,  250143,  228211, 1090033,  982622,  177977,  700660,
        152160,  584462,  415188, 1065438,  161551,   98046,  305424,
        601585,   86080,  209041,  816187,  820975,  158124,  954360,
        227984,  282807,  174036,  229730], dtype=int32)

In [ ]:
prng = jax.random.PRNGKey(1337)

def get_batch(split, prng):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    prng, subkey = random.split(prng)
    ix = random.randint(subkey, (batch_size,), 0, len(data) - block_size)
    print(ix)
    x = jnp.stack([data[i:i+block_size] for i in ix])
    y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = device_put(x), device_put(y)
    return x, y
get_batch('eval', prng)

[ 39216  10190  43768  52429  59508  23294  90087  57102   1663  19305
  39405  11300  16166 110476  99338  98527  32447  17017     25  48318
  88658  19627  26638  97044  76053  46584  90512  13184  64779  20642
  66664  62260]


(Array([[ 1, 56, 39, ..., 23, 52, 53],
        [46, 43, 52, ..., 39, 52,  1],
        [58,  1, 58, ...,  1, 39, 58],
        ...,
        [52, 42,  1, ..., 50, 50,  1],
        [ 0, 14, 21, ..., 58, 53,  1],
        [57,  1, 40, ..., 58,  7, 54]], dtype=int32),
 Array([[56, 39, 58, ..., 52, 53, 61],
        [43, 52,  1, ..., 52,  1, 57],
        [ 1, 58, 46, ..., 39, 58,  1],
        ...,
        [42,  1, 58, ..., 50,  1, 40],
        [14, 21, 13, ..., 53,  1, 24],
        [ 1, 40, 43, ...,  7, 54, 50]], dtype=int32))

#### put together

In [ ]:
prng = jax.random.PRNGKey(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 500
eval_interval = 100
learning_rate = 1e-3
eval_iters = 10
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = jnp.array(encode(text), dtype=jnp.int32)
data = device_put(data)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split, subkey):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    t1 = time.time()
    ix = random.randint(subkey, (batch_size,), 0, len(data) - block_size)
    t2 = time.time()


    # x = jnp.stack([data[i:i+block_size] for i in ix])
    # y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    def slice_data(i):
        return jax.lax.dynamic_slice(data, (i,), (block_size,))

    x = jax.vmap(slice_data)(ix)
    y = jax.vmap(slice_data)(ix+1)
    x, y = device_put(x), device_put(y)
    # print('TIME rand idx', t2-t1)
    # print('TIME rand idx fetch', time.time()-t2)
    return x, y

def estimate_loss(params, prng):
    out = {}
    for split in ['train', 'val']:
        losses = jnp.zeros(eval_iters)
        for k in range(eval_iters):
            prng, subkey = random.split(prng)
            X, Y = get_batch(split, subkey)
            logits, loss = model.apply(params, X, Y)
            losses = losses.at[k].set(loss)
        out[split] = losses.mean()
    return out

class BigramLanguageModel(nn.Module):
    vocab_size: int

    @nn.compact
    def __call__(self, idx, targets=None):
        # Token embedding table
        embedding_table = nn.Embed(num_embeddings=self.vocab_size, features=self.vocab_size)
        logits = embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = -jnp.sum(jax.nn.one_hot(targets, C) * jax.nn.log_softmax(logits), axis=1).mean()

        return logits, loss

    def generate(self, params, key, idx, max_new_tokens):
      for _ in range(max_new_tokens):
          logits, _ = self.apply(params, idx)
          logits = logits[:, -1, :]  # (B, C)
          # probs = jax.nn.softmax(logits, axis=-1)  # (B, C)
          key, subkey = random.split(key)
          idx_next = random.categorical(subkey, logits)[:, None]  # (B, 1)
          idx = jnp.concatenate((idx, idx_next), axis=1)  # (B, T+1)
      return idx

# Define the optimizer
tx = adam(learning_rate)

# Initialize model parameters and optimizer state
model = BigramLanguageModel(vocab_size)

# Initialize model parameters and optimizer
params = model.init(prng, jnp.ones((1, 1), jnp.int32))
opt_state = tx.init(params)

# Loss function (assuming you have a batch of data: xb, yb)
def loss_fn(params, xb, yb):
    logits, loss = model.apply(params, xb, yb)
    return loss

# Update function for a single training step
@jax.jit
def update_step(params, opt_state, xb, yb):
    loss, grads = jax.value_and_grad(loss_fn)(params, xb, yb)
    updates, opt_state = tx.update(grads, opt_state, params)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state, loss

# Training loop (example)
batch_size = 32
t = time.time()
for steps in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    t1 = time.time()
    prng, subkey = random.split(prng)
    xb, yb = get_batch('train', subkey)
    t2 = time.time()
    params, opt_state, loss = update_step(params, opt_state, xb, yb)
    print(f"Epoch {steps},Loss: {loss}")
print('TIME jax', time.time()-t)

Epoch 0,Loss: 4.176700115203857
Epoch 1,Loss: 4.168147563934326
Epoch 2,Loss: 4.17624568939209
Epoch 3,Loss: 4.169358253479004
Epoch 4,Loss: 4.175686359405518
Epoch 5,Loss: 4.170393943786621
Epoch 6,Loss: 4.169347763061523
Epoch 7,Loss: 4.162715911865234
Epoch 8,Loss: 4.157233238220215
Epoch 9,Loss: 4.163628101348877
Epoch 10,Loss: 4.16217565536499
Epoch 11,Loss: 4.16450309753418
Epoch 12,Loss: 4.15981388092041
Epoch 13,Loss: 4.157349586486816
Epoch 14,Loss: 4.163050174713135
Epoch 15,Loss: 4.155461311340332
Epoch 16,Loss: 4.154058456420898
Epoch 17,Loss: 4.14748477935791
Epoch 18,Loss: 4.1531476974487305
Epoch 19,Loss: 4.15305757522583
Epoch 20,Loss: 4.148075580596924
Epoch 21,Loss: 4.1545729637146
Epoch 22,Loss: 4.145484924316406
Epoch 23,Loss: 4.146124839782715
Epoch 24,Loss: 4.152686595916748
Epoch 25,Loss: 4.148232460021973
Epoch 26,Loss: 4.143317222595215
Epoch 27,Loss: 4.135959625244141
Epoch 28,Loss: 4.133705139160156
Epoch 29,Loss: 4.144227981567383
Epoch 30,Loss: 4.1371746063

## The mathematical trick in self-attention

In [ ]:
import jax
import jax.numpy as jnp
from jax import random

# set the random key and seed
key = random.PRNGKey(42)

# create a lower triangular matrix
a = jnp.tril(jnp.ones((3, 3)))
print(a)
a = a / jnp.sum(a, axis=1, keepdims=True)
print(a)
# create a random matrix
b = random.randint(key, (3, 2), 0, 10, dtype=jnp.int32)

# perform the matrix multiplication
c = jnp.matmul(a, b)

# print the matrices and the result
print("a=")
print(a)
print("--")
print("b=")
print(b)
print("--")
print("c=")
print(c)

[[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]]
[[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333334 0.33333334 0.33333334]]
a=
[[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333334 0.33333334 0.33333334]]
--
b=
[[2 3]
 [9 9]
 [4 6]]
--
c=
[[2.  3. ]
 [5.5 6. ]
 [5.  6. ]]


In [ ]:
# Set the random key and seed
key = random.PRNGKey(1337)

# Define the batch size, sequence length, and number of channels
B, T, C = 4, 8, 2

# Generate a random tensor with the specified shape
x = random.normal(key, (B, T, C))

# Print the shape of the tensor
print(x.shape)

(4, 8, 2)


In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
# Initialize xbow tensor
xbow = jnp.zeros((B, T, C))

# Loop over batch and time dimensions
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t, C)
        mean_xprev = jnp.mean(xprev, axis=0)
        xbow = xbow.at[b, t].set(mean_xprev)

# Print the result
print(xbow)

[[[ 1.3654243  -1.3698599 ]
  [ 1.8501079  -1.18029   ]
  [ 1.2796469  -1.1314313 ]
  [ 1.2234701  -0.95123225]
  [ 1.0816249  -0.8464665 ]
  [ 0.9756752  -0.40811488]
  [ 1.089724   -0.345884  ]
  [ 1.0498141  -0.29275966]]

 [[-0.02413951  1.4920624 ]
  [-0.37343726  1.7603257 ]
  [-0.48404503  1.4237347 ]
  [-0.18112068  1.2025388 ]
  [-0.28402337  0.9092719 ]
  [-0.21978617  0.8809384 ]
  [-0.3614355   1.045876  ]
  [-0.27126977  0.8439261 ]]

 [[ 0.67737067  0.45489657]
  [-0.02651259  0.6725235 ]
  [ 0.3897322   0.32223004]
  [-0.05313486  0.49777415]
  [-0.09553531  0.17514856]
  [ 0.09583326  0.24674284]
  [ 0.17541157  0.14904368]
  [ 0.36874425 -0.14085238]]

 [[-1.1729926  -1.0436211 ]
  [-0.76886547 -0.83241093]
  [-0.1617876  -0.5863479 ]
  [-0.23202893 -0.7893201 ]
  [-0.09417699 -0.38362506]
  [-0.03415637 -0.40888947]
  [-0.10969827 -0.27693108]
  [ 0.08133804 -0.42021018]]]


In [ ]:
# version 2: using matrix multiply for a weighted aggregation

# Create a lower triangular matrix with normalized rows
wei = jnp.tril(jnp.ones((T, T)))
wei = wei / jnp.sum(wei, axis=1, keepdims=True)

# Perform the weighted aggregation using matrix multiplication
xbow2 = jnp.matmul(wei, x)  # (T, T) @ (B, T, C) ----> (B, T, C)

# Check if the results are close
all_close = jnp.allclose(xbow, xbow2)
print(all_close)

True


In [ ]:
wei = jnp.tril(jnp.ones((T, T)))
wei = wei / jnp.sum(wei, axis=1, keepdims=True)
wei

Array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.5       , 0.5       , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.25      , 0.25      , 0.25      , 0.25      , 0.        ,
        0.        , 0.        , 0.        ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
        0.        , 0.        , 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        , 0.        ],
       [0.14285715, 0.14285715, 0.14285715, 0.14285715, 0.14285715,
        0.14285715, 0.14285715, 0.        ],
       [0.125     , 0.125     , 0.125     , 0.125     , 0.125     ,
        0.125     , 0.125     , 0.125     ]], dtype=float32)

In [ ]:
# version 3: use Softmax
from jax.nn import softmax

# Create a lower triangular mask
tril = jnp.tril(jnp.ones((T, T)))

# Create a mask filled with negative infinity where tril is 0
wei = jnp.zeros((T, T))
wei = jnp.where(tril == 0, -jnp.inf, wei)

# Apply softmax along the last dimension
wei = jnn.softmax(wei, axis=-1)

# Perform the weighted aggregation using matrix multiplication
xbow3 = jnp.matmul(wei, x)

# Check if the results are close
all_close = jnp.allclose(xbow, xbow3)
print(all_close)


True


In [ ]:
# Set the random key and seed
rng = random.PRNGKey(1337)

# Create a random input tensor
B,T,C = 4,8,32 # batch, time, channels
x = random.normal(rng, (B, T, C))

# Define the head size for self-attention
head_size = 16

# Define the linear layers for key, query, and value
key = nn.Dense(head_size, kernel_init=nn.initializers.glorot_normal())
query = nn.Dense(head_size, kernel_init=nn.initializers.glorot_normal())
value = nn.Dense(head_size, kernel_init=nn.initializers.glorot_normal())

# Compute the key, query, and value projections
k_variables = key.init(rng, random.normal(rng, (B, T, C)))
print(x.shape, k_variables['params']['kernel'].shape)

k = key.apply(k_variables, x) # (B, T, 16)
print(k.shape)

q_variables = query.init(rng, x)
q = query.apply(q_variables, x) # (B, T, 16)
print(q.shape)

# Compute the attention weights
wei = jnp.matmul(q, k.transpose((0, 2, 1)))  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
# Create a lower triangular mask
tril = jnp.tril(jnp.ones((T, T)))

# Apply the mask and then softmax along the last dimension
wei = jnp.where(tril == 0, -jnp.inf, wei)
wei = nn.softmax(wei, axis=-1)
print(wei.shape, wei[0].shape)

# Compute the output using the attention weights and the value projection
v_variables = value.init(rng, x)
v = value.apply(v_variables, x)
out = jnp.matmul(wei, v)

# Print the shape of the output
print(out.shape)


(4, 8, 32) (32, 16)
(4, 8, 16)
(4, 8, 16)
(4, 8, 8) (8, 8)
(4, 8, 16)


In [ ]:
wei[0], wei.shape

(Array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [9.7633256e-03, 9.9023670e-01, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [3.8466230e-01, 3.2870498e-01, 2.8663272e-01, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0245054e-01, 8.3960545e-01, 7.0171729e-03, 5.0926875e-02,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [2.0860654e-01, 5.2596384e-01, 5.7231013e-02, 1.3179399e-01,
         7.6404594e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [4.4360008e-02, 2.5677064e-01, 1.2710307e-02, 9.1593184e-02,
         4.3377329e-02, 5.5118859e-01, 0.0000000e+00, 0.0000000e+00],
        [1.7478675e-02, 8.2577473e-01, 1.8446613e-04, 7.8453608e-03,
         9.8771520e-04, 1.8481736e-03, 1.4588076e-01, 0.0000000e+00],
        [8.8755831e-02, 4.8

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:

# Create random key
key = random.PRNGKey(1337)

# Create random query and key tensors
q = random.normal(key, (B, T, head_size))
k = random.normal(key, (B, T, head_size))

# Compute the dot product between q and k and scale by head_size^-0.5
wei = jnp.matmul(q, k.transpose((0, 2, 1))) * head_size**-0.5

# Print the shape of the output
print(wei.shape)

(4, 8, 8)


In [ ]:
k.var()

Array(0.91944367, dtype=float32)

In [ ]:
q.var()

Array(0.91944367, dtype=float32)

In [ ]:
wei.var()

Array(2.3990507, dtype=float32)

In [ ]:
nn.softmax(jnp.array([0.1, -0.2, 0.3, -0.2, 0.5]))

Array([0.1924978 , 0.14260589, 0.23511736, 0.14260589, 0.287173  ],      dtype=float32)

In [ ]:
nn.softmax(jnp.array([0.1, -0.2, 0.3, -0.2, 0.5])*8) # gets too peaky, converges to one-hot

Array([0.03260834, 0.00295816, 0.16151018, 0.00295816, 0.79996514],      dtype=float32)

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = jnp.ones((dim,))
        self.beta = jnp.zeros((dim,))

    def __call__(self, x):
        # Calculate the forward pass
        xmean = x.mean(1, keepdims=True) # batch mean
        xvar = x.var(1, keepdims=True)   # batch variance
        xhat = (x - xmean) / jnp.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

# Set the random key and seed
rng = random.PRNGKey(1337)

# Create an instance of LayerNorm1d
module = LayerNorm1d(100)

# Create a random input tensor
x = random.normal(rng, (32, 100)) # batch size 32 of 100-dimensional vectors

# Apply the LayerNorm1d module to the input tensor
x = module(x)

# Print the shape of the output tensor
print(x.shape)


(32, 100)


In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(Array(-0.03567989, dtype=float32), Array(0.9839538, dtype=float32))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(Array(-7.152557e-09, dtype=float32), Array(0.99999595, dtype=float32))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, random

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------


# random number generator
rng = random.PRNGKey(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = jnp.array(encode(text), dtype=jnp.int32)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = random.randint(rng, (batch_size,), 0, len(data) - block_size)
    x = jnp.stack([data[i:i+block_size] for i in ix])
    y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.device_put(device), y.device_put(device)
    return x, y

@jax.jit
def estimate_loss():
    out = {}
    for split in ['train', 'val']:
        losses = jnp.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model.apply(params, X, Y)
            losses = jax.ops.index_update(losses, k, loss)
        out[split] = losses.mean()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tril = jnp.tril(jnp.ones((block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = jnp.matmul(q, k.transpose((0,2,1))) * C**-0.5 # (B, T, T)
        wei = jax.ops.index_update(wei, self.tril[:T, :T] == 0, -1e9) # (B, T, T)
        wei = jax.nn.softmax(wei, axis=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = jnp.matmul(wei, v) # (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = jnp.concatenate([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(jnp.arange(T)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = jnp.mean(jax.nn.log_softmax(logits, axis=-1)[jnp.arange(B*T), targets]) * -1

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = jax.nn.softmax(logits, axis=-1) # (B, C)
            # sample from the distribution
            idx_next = jax.random.categorical(key, probs, axis=-1) # (B, 1)
            # append sampled index to the running sequence
            idx = jnp.concatenate((idx, idx_next), axis=1) # (B, T+1)
        return idx

# Define the optimizer
tx = adam(learning_rate)

# Initialize model parameters and optimizer state
rng = random.PRNGKey(1337)
params = model.init(rng, jnp.ones((B, T, C), jnp.int32))
opt_state = tx.init(params)

# Loss function (assuming you have a batch of data: xb, yb)
def loss_fn(params, xb, yb):
    print(xb, yb)
    logits, loss = model.apply(params, xb, yb)
    return loss

# Update function for a single training step
@jax.jit
def update_step(params, opt_state, xb, yb):
    loss, grads = jax.value_and_grad(loss_fn)(params, xb, yb)
    updates, opt_state = tx.update(grads, opt_state, params)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state, loss

# Training loop (example)
batch_size = 32
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    params, opt_state, loss = update_step(params, opt_state, xb, yb)
    print(f"Epoch {steps},Loss: {loss}")

model = BigramLanguageModel(vocab_size)

# create a JAX optimizer
optimizer = jax.experimental.optimizers.adamw(learning_rate=learning_rate).create(model.parameters())

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    gradients = grad(lambda params, x, y: model(x, y)[1])(optimizer.target, xb, yb)
    optimizer = optimizer.apply_gradient(gradients)

# generate from the model
context = jnp.zeros((1, 1), dtype=jnp.int32)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


AttributeError: module 'flax.linen' has no attribute 'Embedding'